<a href="https://colab.research.google.com/github/fadodo/Satellite_image_segmentation/blob/main/Semantic_Segmentation_of_Satellite_Imagery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Segmentation of Satellite Imagery using CNNs and SAM


The goal of this Google Colab project is to accurately automatically segment a satellite imagery.

To achieve this, a Convolutional Neural Network (CNN) is employed in conjunction with the [Segment Anything Model (SAM)](https://segment-anything.com/), a sophisticated segmentation algorithm known for its strong performance.

This Python-based implementation utilizes the leafmap library to handle and process the geospatial data, allowing users to select specific regions of interest for analysis. The algorithm has demonstrated its effectiveness in segmenting the differents objects mainly buildings.


```
# Make sure you use GPU runtime for this notebook. For Google Colab, go to Runtime -> Change runtime type and select GPU as the hardware accelerator.
```


# Install the necessaries libraries and packages if not yet installed

`!pip list` to check installed packages

In [ ]:
!pip install leafmap segment-geospatial groundingdino-py localtileserver -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 10.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 537.2/537.2 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.8/87.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [ ]:
# import the necessaries packages

import os
import leafmap
from groundingdino.util.slconfig import SLConfig # Import SLConfig
from samgeo import SamGeo, tms_to_geotiff, get_basemaps

Create an interactive map with leafmap

In [ ]:
# Leafmap for visualization
map=leafmap.Map(zoom=19)
map

Get automaticaly the region of interest coordinates by drawing a box on the map

In [ ]:
# Get the Area of Interest box coordinates
bbox=map.user_roi_bounds()
bbox

[39.2751, -6.8128, 39.2775, -6.8106]

In [ ]:
# You can also manually define the ROI coordinate as follow
# Bbox on a small part of Dar El Salaam
bbox = [39.2751, -6.8128, 39.2775, -6.8106]

Downlading the satellite images of the ROI

In [ ]:
# Downlading the satellite image of the ROI and mosaic it
img="bimg.tif"
tms_to_geotiff(output=img, bbox=bbox, zoom=20, source="SATELLITE", overwrite=True)

Downloaded image 01/56
Downloaded image 02/56
Downloaded image 03/56
Downloaded image 04/56
Downloaded image 05/56
Downloaded image 06/56
Downloaded image 07/56
Downloaded image 08/56
Downloaded image 09/56
Downloaded image 10/56
Downloaded image 11/56
Downloaded image 12/56
Downloaded image 13/56
Downloaded image 14/56
Downloaded image 15/56
Downloaded image 16/56
Downloaded image 17/56
Downloaded image 18/56
Downloaded image 19/56
Downloaded image 20/56
Downloaded image 21/56
Downloaded image 22/56
Downloaded image 23/56
Downloaded image 24/56
Downloaded image 25/56
Downloaded image 26/56
Downloaded image 27/56
Downloaded image 28/56
Downloaded image 29/56
Downloaded image 30/56
Downloaded image 31/56
Downloaded image 32/56
Downloaded image 33/56
Downloaded image 34/56
Downloaded image 35/56
Downloaded image 36/56
Downloaded image 37/56
Downloaded image 38/56
Downloaded image 39/56
Downloaded image 40/56
Downloaded image 41/56
Downloaded image 42/56
Downloaded image 43/56
Downloaded 

Display the downloaded image on the map

In [ ]:
# Adding the satellite mosaic
map.add_raster(img, layer_name="AOImage")
map

### Initialize SAM class¶
The main class for segmenting geospatial data with the Segment Anything Model (SAM).

In [ ]:
# Automatic parameters tunning for mask generation
sam_kwargs = {
    "points_per_side": 32,
    "pred_iou_thresh": 0.86,
    "stability_score_thresh": 0.92,
    "crop_n_layers": 1,
    "crop_n_points_downscale_factor": 2,
    "min_mask_region_area": 100,
}

In [ ]:
# Initialize SAM all class
## There a model checkpoit(default vit_h, vit_l, vit_b)
sam = SamGeo(
    model_type="vit_h",
    checkpoint="sam_vit_h_4b8939.pth",
    sam_kwargs=sam_kwargs,
)

### Segment the image

In [ ]:
# Automatic mask generation
mask="masks.tif"
sam.generate(img, output=mask, batch=True, foreground=True, erosion_kernel=(3, 3), mask_multiplier=255, unique=True)

100%|██████████| 16/16 [06:07<00:00, 23.00s/it]


### Visualize the results

In [ ]:
# Visualize the results
# Show the binary mask or the mask of objects with unique values
sam.show_masks(cmap="binary_r", axis="off", foreground=True)

In [ ]:
# Show the object annotations (objects with random color) on the map
sam.show_anns(axis="off", alpha=1, output="annotations.tif", blend=True)

Please run generate() first.


### Comaprison map with a slider

In [ ]:
# Comparison map to check the efficiency of the detection
leafmap.image_comparison(
    'bimg.tif',
    'masks.tif',
    label1='Satellite image',
    label2='Segment image',
)

### Polygonize the raster data  and display on the map

In [ ]:
## vectorisation
shapefile = "segment.shp"
sam.tiff_to_vector(mask, shapefile)

In [ ]:
## Display the vector on the map
map.add_vector(shapefile, layer_name='Vector')

In [ ]:
map